In [79]:
import pandas as pd
# import mysql.connector import connect
df = pd.read_csv('kindle_data-v2.csv')
df = df.dropna()
df = df.drop_duplicates()
df = df.drop(columns = ['isEditorsPick', 'isGoodReadsChoice', 'isKindleUnlimited','asin'],axis =1)
df.head()

,title,author,soldBy,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,publishedDate,category_name
0,Adult Children of Emotionally Immature Parents...,Lindsay C. Gibson,Amazon.com Services LLC,https://m.media-amazon.com/images/I/713KZTsaYp...,https://www.amazon.com/dp/B00TZE87S4,4.8,0,9.99,6,True,2015-06-01,Parenting & Relationships
1,"From Strength to Strength: Finding Success, Ha...",Arthur C. Brooks,Penguin Group (USA) LLC,https://m.media-amazon.com/images/I/A1LZcJFs9E...,https://www.amazon.com/dp/B08WCKY8MB,4.4,0,16.99,6,False,2022-02-15,Parenting & Relationships
2,Good Inside: A Guide to Becoming the Parent Yo...,Becky Kennedy,HarperCollins Publishers,https://m.media-amazon.com/images/I/71RIWM0sv6...,https://www.amazon.com/dp/B09KPS84CJ,4.8,0,16.99,6,False,2022-09-13,Parenting & Relationships
3,Everything I Know About Love: A Memoir,Dolly Alderton,HarperCollins Publishers,https://m.media-amazon.com/images/I/71QdQpTiKZ...,https://www.amazon.com/dp/B07S7QPG6J,4.2,0,9.95,6,False,2020-02-25,Parenting & Relationships
4,The Seven Principles for Making Marriage Work:...,John Gottman,Random House LLC,https://m.media-amazon.com/images/I/813o4WOs+w...,https://www.amazon.com/dp/B00N6PEQV0,4.7,0,13.99,6,False,2015-05-05,Parenting & Relationships


In [80]:

from mysql.connector import connect
from mysql.connector import Error
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import re
warnings.filterwarnings("ignore")   

In [81]:
conn = connect(
    host = 'localhost',
    user = 'root',
    password = '123456',
    database = 'assignment',
    port = 3306
)
conn.is_connected()

True

In [82]:
sql = """
select * from book
"""
pd.read_sql(sql, conn)

,Book_id,Title,Price,Language,Availability_Status,Publisher_id,Author_ID,Category_id,Amount,PublishDate,img_url


In [83]:
sql = """
select * from author
"""
pd.read_sql(sql, conn)

,Author_id,Name,Nationality


In [84]:
# sql = """
# select * from book_author
# """
# pd.read_sql(sql, conn)

In [85]:
# sql = """
# select * from book_category
# # """
# pd.read_sql(sql, conn)


In [86]:
sql = """
select * from publisher
"""
pd.read_sql(sql, conn)

,Publisher_id,Name,Address,Contact_info


In [87]:
sql = """
select * from category
"""
pd.read_sql(sql, conn)


,Category_id,Category_Name,Description


In [88]:
ebook_categories = {
    0: "Expert advice and strategies for raising children, from infant care to teen parenting",
    1: "Guides for building and maintaining healthy personal and romantic relationships",
    2: "Technical books covering various engineering disciplines and practical applications",
    3: "Resources on vehicles, traffic systems, and transportation infrastructure",
    4: "Books exploring scientific principles, discoveries, and research across disciplines",
    5: "Materials covering mathematical concepts, from basic arithmetic to advanced theory",
    6: "Intriguing stories focused on solving crimes, puzzles, or uncovering secrets",
    7: "Suspenseful fiction featuring intense plot-driven narratives and high stakes",
    8: "Books built around tension and anticipation, often with psychological elements",
    9: "Guides for various handicrafts, DIY projects, and artistic creations",
    10: "Resources for leisure activities, collecting, and recreational pursuits",
    11: "Books about home maintenance, decoration, and domestic management",
    12: "Works studying the art of written expression and literary analysis",
    13: "Imaginative storytelling across various genres and styles",
    14: "Accounts of notable individuals' lives written by others",
    15: "Personal life stories and experiences written by the subjects themselves",
    16: "Technical books about computer systems, programming, and digital technology",
    17: "Materials covering technological advances, innovations, and their applications",
    18: "Books examining political systems, governance, and current affairs",
    19: "Studies of human society, behavior, and social relationships",
    20: "Content specifically written for readers aged 13-19",
    21: "Books targeting readers in their late teens and early twenties",
    22: "Resources for learning and studying non-English languages",
    23: "Fiction exploring futuristic concepts, technology, and space exploration",
    24: "Stories featuring magical elements, mythical creatures, and imaginary worlds",
    25: "Guides, travelogues, and information about destinations worldwide",
    26: "Books focusing on LGBTQ+ themes, experiences, and perspectives",
    27: "Works exploring various religious beliefs, practices, and traditions",
    28: "Books about personal spiritual growth and metaphysical topics",
    29: "Resources covering various forms of artistic expression and creativity",
    30: "Books about photographic techniques, equipment, and artistic vision",
    31: "Graphic novels, manga, and illustrated storytelling",
    32: "Materials about learning theory, educational methods, and academic subjects",
    33: "Resources for educators on instruction methods and classroom management",
    34: "Books examining past events, cultures, and their impact on society",
    35: "Collections of recipes and cooking instructions",
    36: "Books about cuisine, ingredients, and culinary arts",
    37: "Guides to wine appreciation, selection, and pairing",
    38: "Factual books covering real-world topics and information",
    39: "Books focused on personal development and life improvement",
    40: "Stories centered on romantic relationships and love",
    41: "Books intended to entertain and amuse readers",
    42: "Content about media, performances, and popular culture",
    43: "Informational resources and educational materials for reference",
    44: "Books about maintaining and improving physical and mental health",
    45: "Guides for exercise, training, and physical conditioning",
    46: "Resources for weight management and nutritional planning",
    47: "Books covering athletic activities, games, and competition",
    48: "Guides for outdoor activities, nature, and adventure",
    49: "Books about health conditions, treatments, and medical science",
    50: "Resources for business management and entrepreneurship",
    51: "Guides for personal finance, investing, and wealth management",
    52: "Books about legal systems, regulations, and justice"
}

In [89]:
publisher = df['soldBy'].drop_duplicates().to_list()
len(publisher)


37

In [90]:
publisher_contacts = {
    "Amazon.com Services LLC": {
        "address": "410 Terry Ave N, Seattle, WA 98109, United States",
        "email": "publisher.services@amazon.com"
    },
    "Penguin Group (USA) LLC": {
        "address": "475 Hudson Street, New York, NY 10014, United States",
        "email": "publishing@penguinrandomhouse.com"
    },
    "HarperCollins Publishers": {
        "address": "195 Broadway, New York, NY 10007, United States",
        "email": "publisher.info@harpercollins.com"
    },
    "Random House LLC": {
        "address": "1745 Broadway, New York, NY 10019, United States",
        "email": "publishing@randomhouse.com"
    },
    "Simon and Schuster Digital Sales Inc": {
        "address": "1230 Avenue of the Americas, New York, NY 10020, United States",
        "email": "digital.sales@simonandschuster.com"
    },
    "Macmillan": {
        "address": "120 Broadway, New York, NY 10271, United States",
        "email": "publisher@macmillan.com"
    },
    "Simon & Schuster Digital Sales Inc.": {
        "address": "1230 Avenue of the Americas, New York, NY 10020, United States",
        "email": "digital@simonandschuster.com"
    },
    "Hachette Book Group": {
        "address": "1290 Avenue of the Americas, New York, NY 10104, United States",
        "email": "publishing@hachettebookgroup.com"
    },
    "HarperCollins Publishing": {
        "address": "195 Broadway, New York, NY 10007, United States",
        "email": "publishing@harpercollins.com"
    },
    "De Marque": {
        "address": "395 Rue Victor-Revillon, Quebec City, QC G1K 3M8, Canada",
        "email": "contact@demarque.com"
    },
    "W. W. Norton & Company": {
        "address": "500 Fifth Avenue, New York, NY 10110, United States",
        "email": "publishing@wwnorton.com"
    },
    "Penguin Random House Publisher Services": {
        "address": "1745 Broadway, New York, NY 10019, United States",
        "email": "services@penguinrandomhouse.com"
    },
    "Pearson Education,Inc.": {
        "address": "221 River Street, Hoboken, NJ 07030, United States",
        "email": "education@pearson.com"
    },
    "JOHN WILEY AND SONS INC": {
        "address": "111 River Street, Hoboken, NJ 07030, United States",
        "email": "publisher@wiley.com"
    },
    "Harlequin Digital Sales Corp.": {
        "address": "22 Adelaide Street West, Toronto, ON M5H 4E3, Canada",
        "email": "digital.sales@harlequin.com"
    },
    "PRH UK": {
        "address": "20 Vauxhall Bridge Road, London SW1V 2SA, United Kingdom",
        "email": "publishing@penguinrandomhouse.co.uk"
    },
    "RH AU": {
        "address": "Level 3, 100 Pacific Highway, North Sydney, NSW 2060, Australia",
        "email": "info@randomhouse.com.au"
    },
    "Penguin Random House Australia Pty Ltd": {
        "address": "707 Collins Street, Melbourne, VIC 3008, Australia",
        "email": "publishing@penguinrandomhouse.com.au"
    },
    "Cengage Learning": {
        "address": "200 Pier 4 Boulevard, Boston, MA 02210, United States",
        "email": "education@cengage.com"
    },
    "Disney Book Group": {
        "address": "1200 Grand Central Avenue, Glendale, CA 91201, United States",
        "email": "publishing@disney.com"
    },
    "Random House NZ": {
        "address": "67 Apollo Drive, Auckland 0632, New Zealand",
        "email": "publishing@randomhouse.co.nz"
    },
    "Amazon.com": {
        "address": "410 Terry Ave N, Seattle, WA 98109, United States",
        "email": "publishing@amazon.com"
    },
    "Random House India": {
        "address": "423 Udyog Vihar, Phase IV, Gurugram 122016, India",
        "email": "publishing@randomhouse.co.in"
    },
    "Macmillan Higher Education": {
        "address": "1 New York Plaza, New York, NY 10004, United States",
        "email": "education@macmillan.com"
    },
    "EDIGITA": {
        "address": "Via Stefano Turr 38, 20121 Milan, Italy",
        "email": "publishing@edigita.it"
    },
    "Gallimard Lt.": {
        "address": "5 Rue Gaston-Gallimard, 75007 Paris, France",
        "email": "edition@gallimard.fr"
    },
    "Flammarion Lt.": {
        "address": "87 Quai Panhard et Levassor, 75013 Paris, France",
        "email": "edition@flammarion.fr"
    },
    "Random House ZA": {
        "address": "Isle of Houghton, Corner Boundary & Carse O'Gowrie Roads, Johannesburg 2198, South Africa",
        "email": "publishing@randomhouse.co.za"
    },
    "Games Workshop": {
        "address": "Willow Road, Lenton, Nottingham NG7 2WS, United Kingdom",
        "email": "publishing@gwplc.com"
    },
    "Amazon Digital Services LLC HN": {
        "address": "410 Terry Ave N, Seattle, WA 98109, United States",
        "email": "digital.services.hn@amazon.com"
    },
    "Scholastic Trade Publisher": {
        "address": "557 Broadway, New York, NY 10012, United States",
        "email": "trade@scholastic.com"
    },
    "Yen Press LLC": {
        "address": "1290 Avenue of the Americas, New York, NY 10104, United States",
        "email": "publishing@yenpress.com"
    },
    "Amazon Digital Services LLC GU": {
        "address": "410 Terry Ave N, Seattle, WA 98109, United States",
        "email": "digital.services.gu@amazon.com"
    },
    "Amazon Digital Services LLC MK": {
        "address": "410 Terry Ave N, Seattle, WA 98109, United States",
        "email": "digital.services.mk@amazon.com"
    },
    "Immat&eacute;riel fr": {
        "address": "12 Rue de la Chaussée d'Antin, 75009 Paris, France",
        "email": "contact@immateriel.fr"
    },
    "Pottermore": {
        "address": "27 Wrights Lane, London W8 5SW, United Kingdom",
        "email": "publishing@pottermore.com"
    },
    "Marvel Entertainment US": {
        "address": "10 East 40th Street, New York, NY 10016, United States",
        "email": "publishing@marvel.com"
    }
}

In [91]:
category_df = df['category_name'].str.split('&').explode().reset_index(drop=True)
category = category_df.drop_duplicates().reset_index(drop=True)
category = category.str.split(',').explode().reset_index(drop=True)
category = category.drop_duplicates().reset_index(drop=True)
category = category.str.strip()
category[14]

'Biographies'

In [92]:
author = df['author'].drop_duplicates().reset_index(drop=True)
author = author.str.split(',').explode().reset_index(drop=True)
author = author.drop_duplicates().reset_index(drop=True)
author = author.str.strip()
author = author[0:100].to_list()
# author


In [93]:
author_nationalities = {
    'Lindsay C. Gibson': 'American',
    'Arthur C. Brooks': 'American',
    'Becky Kennedy': 'American',
    'Dolly Alderton': 'British',
    'John Gottman': 'American',
    'Jeannette Walls': 'American',
    'Jennifer Breheny Wallace': 'American',
    'Devon Price': 'American',
    'Oprah Winfrey': 'American',
    'Andy Cohen': 'American',
    'Caroline James': 'British',
    'Maggie Smith': 'American',
    'Daniel J. Siegel': 'American',
    'Michael Brodkorb': 'American',
    'Robert Glover': 'American',
    'James McBride': 'American',
    'Jessica Fern': 'American',
    'Robert F. Kennedy Jr.': 'American',
    'Lundy Bancroft': 'American',
    'Michaeleen Doucleff': 'American',
    'David Sedaris': 'American',
    'Stephen King': 'American',
    'Gabor Maté': 'Hungarian-Canadian',
    'Taylor Wolfe': 'American',
    'K.L. Randis': 'American',
    'Alexis Dubief': 'American',
    'Ross W. Greene': 'American',
    'Ryan Holiday': 'American',
    'Justin Whitmel Earley': 'American',
    'Edward M. Hallowell': 'American',
    'Lisa Damour Ph.D.': 'American',
    'Pamela Druckerman': 'American',
    'Esther Perel': 'Belgian',
    'Ryder Carroll': 'American',
    'Hunter Clarke-Fields MSAE': 'American',
    'Miriam Grossman': 'American',
    'Brant Hansen': 'American',
    'L. M. Montgomery': 'Canadian',
    'Cathy Glass': 'British',
    'Gabor Maté M.D.': 'Hungarian-Canadian',
    'Lily Nichols': 'American',
    'Morgan Cutlip': 'American',
    'Caitlin Moran': 'British',
    'Daniel Hurst': 'British',
    'Nedra Glover Tawwab': 'American',
    'Jessica Willis Fisher': 'American',
    'David Crow': 'American',
    'Jancee Dunn': 'American',
    'Andrew McCarthy': 'American',
    'Catherine Gray': 'British',
    'Bruce D Perry': 'American',
    'Luke Russert': 'American',
    'Emerson Eggerichs PhD': 'American',
    'Anne Fadiman': 'American',
    'Terrence Real': 'American',
    'Carmen Maria Machado': 'American',
    'Gordon Neufeld': 'Canadian',
    'Olivia Fox Cabane': 'French-American',
    'Elena Collins': 'British',
    'Sissy Goff': 'American',
    'Mira Kirshenbaum': 'American',
    'Heidi Priebe': 'Canadian',
    'Lysa TerKeurst': 'American',
    'Richard Pink': 'British',
    'Rachael English': 'Irish',
    'Maria Tatar': 'American',
    'James J. Sexton': 'American',
    'Amanda Montei': 'American',
    'Shirley Glass': 'American',
    'Caroline Fraser': 'American',
    'Joshua Coleman Ph D.': 'American',
    'Philippa Perry': 'British',
    'Ramani Durvasula': 'American',
    'Judith Lewis Herman MD': 'American',
    'Mary Louise Kelly': 'American',
    'Leslie Jordan': 'American',
    'Teresa M. McDevitt': 'American',
    'Sherry Argov': 'American',
    'Tamara Phd Rosier': 'American',
    'Alex Lake': 'British',
    'Toni Weschler': 'American',
    'Vanessa Garbin': 'Australian',
    'LICSW Resmaa Menakem': 'American',
    'MSW': 'Unknown',
    'Adrienne Brodeur': 'American',
    'Harvey Karp': 'American',
    'Matthew Fray': 'American',
    'Neil Strauss': 'American',
    'John Bradshaw': 'American',
    'Paul Thomas': 'American',
    'Michael S. Kelly': 'American',
    'Kerri Rawson': 'American',
    'Rowena Bennett': 'Australian',
    'Olivia Pearl': 'American',
    'Brandi Brucks': 'American',
    'Thomas W. Phelan': 'American',
    'Emma-Claire Wilson': 'British',
    'Danya Ruttenberg': 'American',
    'Karyl McBride': 'American',
    'Kerry Patterson': 'American'
}

In [94]:
book = df[['title','price','publishedDate','imgUrl','soldBy','category_name','author','stars']]
book.head(10) 
book= book.reset_index(drop = True)
book

,title,price,publishedDate,imgUrl,soldBy,category_name,author,stars
0,Adult Children of Emotionally Immature Parents...,9.99,2015-06-01,https://m.media-amazon.com/images/I/713KZTsaYp...,Amazon.com Services LLC,Parenting & Relationships,Lindsay C. Gibson,4.8
1,"From Strength to Strength: Finding Success, Ha...",16.99,2022-02-15,https://m.media-amazon.com/images/I/A1LZcJFs9E...,Penguin Group (USA) LLC,Parenting & Relationships,Arthur C. Brooks,4.4
2,Good Inside: A Guide to Becoming the Parent Yo...,16.99,2022-09-13,https://m.media-amazon.com/images/I/71RIWM0sv6...,HarperCollins Publishers,Parenting & Relationships,Becky Kennedy,4.8
3,Everything I Know About Love: A Memoir,9.95,2020-02-25,https://m.media-amazon.com/images/I/71QdQpTiKZ...,HarperCollins Publishers,Parenting & Relationships,Dolly Alderton,4.2
4,The Seven Principles for Making Marriage Work:...,13.99,2015-05-05,https://m.media-amazon.com/images/I/813o4WOs+w...,Random House LLC,Parenting & Relationships,John Gottman,4.7
...,...,...,...,...,...,...,...,...
77795,Anna Karenina,0.00,2023-09-15,https://m.media-amazon.com/images/I/61RXEyMctk...,De Marque,Nonfiction,Leo Tolstoy,4.7
77796,Scary Smart: Scary Smart: The Future of Artifi...,9.99,2021-09-30,https://m.media-amazon.com/images/I/71tjf6R-Wa...,Amazon.com Services LLC,Nonfiction,Mo Gawdat,4.4
77797,The Count of Monte Cristo: The Original Unabri...,1.99,2023-09-15,https://m.media-amazon.com/images/I/71zcCb5Pvu...,De Marque,Nonfiction,Alexandre Dumas,4.8
77798,War And Peace,0.00,2023-08-25,https://m.media-amazon.com/images/I/715QpZlzmi...,De Marque,Nonfiction,Leo Tolstoy,4.7


adding data to Category

In [95]:

for i in range(0, 53):
    sql = """
    INSERT INTO category (category_id, category_name, description) 
    VALUES (%s, %s, %s)
    """
    cursor = conn.cursor()
    values = (i+1, category[i], ebook_categories[i])
    cursor.execute(sql, values)
    conn.commit()

In [96]:
for i in range(0,37):
    sql = """
    INSERT INTO publisher (publisher_id, name, address, contact_info) 
    VALUES (%s, %s, %s, %s)
    """
    cursor = conn.cursor()
    values = (i+1, publisher[i], publisher_contacts[publisher[i]]['address'], publisher_contacts[publisher[i]]['email'])
    cursor.execute(sql, values)
    conn.commit()

adding data to author table 


In [97]:
for i in range(0, len(author)):
    sql = """
    INSERT INTO author (author_id, name, nationality) 
    VALUES (%s, %s, %s)
    """
    cursor = conn.cursor()
    values = (i+1, author[i], author_nationalities[author[i]])
    cursor.execute(sql, values)
    conn.commit()

functions to map to the correct ones


In [98]:

def map_category(category_name):
    result = re.split(r'[^\w]+', category_name)
    result = [x.strip() for x in result]
    category_id = []
    for i in range(0, len(result)):
        sql = """
        SELECT category_id FROM category WHERE category_name = %s
        """
        cursor = conn.cursor()
        cursor.execute(sql, (result[i],))
        category_id.append(cursor.fetchone()[0])
    return ','.join(map(str, category_id))
  


In [99]:
def map_author(author_name):
    sql = """
    SELECT author_id FROM author WHERE name = %s
    """
    cursor = conn.cursor()
    cursor.execute(sql, (author_name,))
    author_id = cursor.fetchone()[0]
    return author_id

In [100]:
def map_publisher(publisher_name):
    sql = """
    SELECT publisher_id FROM publisher WHERE name = %s
    """
    cursor = conn.cursor()
    cursor.execute(sql, (publisher_name,))
    return cursor.fetchone()[0]

adding 80 books into database

In [101]:
import random

for i in range(0, 80):
    sql = """
    INSERT INTO book (book_id, title, price, language, availability_status, publisher_id, author_id, amount, category_id, publishdate, img_url)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor = conn.cursor()
    values = (
        i + 1, 
        book['title'][i], 
        int(round(book['price'][i])), 
        'English', 
        'Available',
        map_publisher(book['soldBy'][i]), 
        map_author(book['author'][i]), 
        random.randint(0, 255),  # Adjusted range
        map_category(book['category_name'][i]), 
        book['publishedDate'][i], 
        book['imgUrl'][i]
    )
    cursor.execute(sql, params=values)
    conn.commit()


In [102]:
for i in range(0,80):
    sql = """
    INSERT INTO reviews (review_id, book_id, rating) 
    VALUES (%s, %s, %s)
    """
    cursor = conn.cursor()
    values = (i+1, i+1, float(book['stars'][i]))
    cursor.execute(sql, values)
    conn.commit()

In [103]:
import random
import string

def generate_coupon_codes(num_codes, length=8):
    codes = set()
    while len(codes) < num_codes:
        code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
        codes.add(code)
    return list(codes)

coupon_codes = generate_coupon_codes(80)
for i in range(0, 80):
    sql = """
    INSERT INTO coupon (coupon_code ,discount_percentage,valid_until)
    VALUES (%s, %s, %s)
    """
    cursor = conn.cursor()
    values = (coupon_codes[i], random.randint(5, 50), '2025-12-31')
    cursor.execute(sql, values)
    conn.commit()


In [104]:
for i in range(0, 80):
    sql = """
    INSERT INTO book_coupon (book_id, coupon_code)
    VALUES (%s, %s)
    """
    random_coupon = random.choice(coupon_codes)
    coupon_codes.remove(random_coupon)
    cursor = conn.cursor()
    values = (i+1, random_coupon)
    
    cursor.execute(sql, values)
    conn.commit()
        